In [25]:
import os
import sys
import psutil
import traci
import time
import math

# Ensure SUMO_HOME is set
if "SUMO_HOME" in os.environ:
    sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

# Function to kill old SUMO processes before starting
def kill_sumo():
    for process in psutil.process_iter(attrs=["pid", "name"]):
        if "sumo" in process.info["name"].lower():
            print(f"Killing SUMO process {process.info['name']} (PID: {process.info['pid']})")
            os.kill(process.info["pid"], 9)

# Check charging stations
def check_charging_stations():
    charging_stations = traci.chargingstation.getIDList()
    for cs in charging_stations:
        lane_id = traci.chargingstation.getLaneID(cs)
        edge_id = traci.lane.getEdgeID(lane_id)
        print(f"Charging Station {cs} is on Edge: {edge_id}")

def park_car(veh_id, parking_area_id, parking_edge):
    """
    Parks the car at a specified parking lot
    :param veh_id: The ID of the vehicle
    :param parking_area_id: The ID of the parking lot
    :param parking_edge: The edge ID of the parking lot
    """
    print(f"Forcing {veh_id} to park at {parking_area_id}.")

    # Get vehicle's current route
    route = traci.vehicle.getRoute(veh_id)
    
    # If the parking area is not on the route, reroute the vehicle
    if parking_edge not in route:
        print(f"WARNING: Parking lot edge {parking_edge} is NOT on {veh_id}'s route! Rerouting...")
        traci.vehicle.changeTarget(veh_id, parking_edge)
        traci.vehicle.rerouteTraveltime(veh_id)
    
    # Ensure the parking stop is set
    try:
        traci.vehicle.setParkingAreaStop(
            vehID=veh_id,
            stopID=parking_area_id,
            duration=300  # Stay parked for 300 seconds
        )
        print(f"{veh_id} is now parking at {parking_area_id}.")
    except traci.TraCIException as e:
        print(f"Error parking {veh_id}: {e}")

def get_parking_lot_vehicles(parking_lot_id):
    parked_vehicle_count = traci.parkingarea.getVehicleCount(parking_lot_id)
    parking_capacity = traci.simulation.getParameter(parking_lot_id, "parkingArea.capacity")
    return parked_vehicle_count, parking_capacity

import traci

def find_parking_along_route(veh_id):
    """
    Finds a parking lot that is along the vehicle's current route.
    :param veh_id: The ID of the vehicle
    :return: parking lot id, edge id, lane id
    """
    # Get the vehicle's current route (list of edges)
    route_edges = traci.vehicle.getRoute(veh_id)

    # Get all parking areas
    parking_areas = traci.parkingarea.getIDList()

    # Find a parking lot that is on the vehicle’s route
    for parking_area_id in parking_areas:
        try:
            lane_id = traci.parkingarea.getLaneID(parking_area_id)
            edge_id = traci.lane.getEdgeID(lane_id)  # Get the edge associated with the parking lot

            if edge_id in route_edges:
                print(f"Found parking area {parking_area_id} along route of {veh_id} on edge {edge_id}.")
                return parking_area_id, edge_id, lane_id

        except traci.TraCIException as e:
            print(f"Error checking parking area {parking_area_id}: {e}")

    print(f"No parking area found along the route of {veh_id}.")
    return None, None, None


def find_closest_parking_lot(veh_id):
    """
    Finds the closest parking lot to the given vehicle.
    :param veh_id: The ID of the vehicle
    :return: Tuple (closest_parking_area_id, closest_edge, distance)
    """
    try:
        # Get vehicle's current position
        veh_x, veh_y = traci.vehicle.getPosition(veh_id)

        closest_parking_area = None
        closest_edge = None
        min_distance = float("inf")

        # Loop through all parking areas
        for parking_area_id in traci.parkingarea.getIDList():
            try:
                # Get the parking lot position (approximate based on lane shape)
                lane_id = traci.parkingarea.getLaneID(parking_area_id)
                edge_id = traci.lane.getEdgeID(lane_id)
                parking_x, parking_y = traci.lane.getShape(lane_id)[0]  # Get first point on lane

                # Calculate Euclidean distance
                distance = math.sqrt((veh_x - parking_x) ** 2 + (veh_y - parking_y) ** 2)

                # Update closest parking area
                if distance < min_distance:
                    min_distance = distance
                    closest_parking_area = parking_area_id
                    closest_edge = edge_id

            except traci.TraCIException as e:
                print(f"Error fetching details for parking area {parking_area_id}: {e}")

        if closest_parking_area:
            print(f"Closest parking area for {veh_id}: {closest_parking_area} on edge {closest_edge} ({min_distance:.2f}m)")
            return closest_parking_area, closest_edge, min_distance
        else:
            print(f"No parking areas found!")
            return None, None, None

    except traci.TraCIException as e:
        print(f"Error finding closest parking for {veh_id}: {e}")
        return None, None, None


In [26]:
kill_sumo()

# Run simulation
sumoBinary = "sumo/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

# Keep track of parked cars to prevent repeated actions
parked_vehicles = set()

try:
    # Increase retries to prevent connection issues
    traci.start(sumoCmd, numRetries=20)
    print("Sumo started successfully.")

    check_charging_stations()

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()

        for veh_id in traci.vehicle.getIDList():
            try:    
                if veh_id in parked_vehicles:
                    continue

                # Battery parameters
                battery_level = float(traci.vehicle.getParameter(veh_id, "device.battery.actualBatteryCapacity"))
                max_battery = float(traci.vehicle.getParameter(veh_id, "device.battery.maximumBatteryCapacity"))

                # Check if stationfinder is enabled
                has_stationfinder = traci.vehicle.getParameter(veh_id, "has.stationfinder.device")

                if battery_level < 2900:
                    parking_area_id, edge_id, lane_id = find_parking_along_route(veh_id)
                    
                    if parking_area_id:
                        park_car(veh_id, parking_area_id, edge_id)
                        parked_vehicles.add(veh_id)  # Mark the vehicle as parked to prevent redoing it
                    else:
                        closest_parking_area, closest_edge, min_distance = find_closest_parking_lot(veh_id)
                        park_car(veh_id, closest_parking_area, closest_edge)
                        parked_vehicles.add(veh_id)  # Mark the vehicle as parked to prevent redoing it

            except traci.TraCIException as e:
                print(f"Error for {veh_id}: {e}")
                
except traci.exceptions.FatalTraCIError as e:
    print(f"TraCI connection lost: {e}")

except BrokenPipeError:
    print("BrokenPipeError: SUMO crashed or closed unexpectedly.")

finally:
    time.sleep(1)
    try:
        if traci.connection._connections:
            print("Closing TraCI connection properly...")
            traci.close()
    except Exception as e:
        print(f"TraCI was already closed: {e}")

 Retrying in 1 seconds
Sumo started successfully.
Charging Station cS_-364911324 is on Edge: -364911324
Charging Station cS_4998703#0 is on Edge: 4998703#0
Charging Station cS_4998713#0 is on Edge: 4998713#0
No parking area found along the route of veh5.
Closest parking area for veh5: ParkAreaC on edge 4998703#0 (154.95m)
Forcing veh5 to park at ParkAreaC.
veh5 is now parking at ParkAreaC.
No parking area found along the route of veh8.
Closest parking area for veh8: ParkAreaA on edge 4998713#0 (145.49m)
Forcing veh8 to park at ParkAreaA.
veh8 is now parking at ParkAreaA.
No parking area found along the route of veh9.
Closest parking area for veh9: ParkAreaC on edge 4998703#0 (89.37m)
Forcing veh9 to park at ParkAreaC.
veh9 is now parking at ParkAreaC.
No parking area found along the route of veh6.
Closest parking area for veh6: ParkAreaA on edge 4998713#0 (272.05m)
Forcing veh6 to park at ParkAreaA.
veh6 is now parking at ParkAreaA.
No parking area found along the route of veh14.
Close